# NYC Taxi Trip Duration

## Module Imports

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import math
from datetime import datetime

In [ ]:
%matplotlib inline
sns.set({'figure.figsize':(16,8), 'axes.titlesize':15, 'axes.labelsize':10})

## Data Loading

In [ ]:
!ls ../input

In [ ]:
FILEPATH = os.path.join("..", "input")
TRAINPATH = os.path.join(FILEPATH, "train.csv")
TESTPATH = os.path.join(FILEPATH, "test.csv")

> Visualisation de l'en-tête du fichier **train.csv**

In [ ]:
df = pd.read_csv(TRAINPATH, index_col=0)
df.head()

> Visualisation de l'en-tête du fichier **test.csv**

In [ ]:
df_test = pd.read_csv(TESTPATH)
df_test.head()

## Data Exploration

### Data fields

`id` - a unique identifier for each trip

`vendor_id` - a code indicating the provider associated with the trip record

`pickup_datetime` - date and time when the meter was engaged

`dropoff_datetime` - date and time when the meter was disengaged

`passenger_count` - the number of passengers in the vehicle (driver entered value)

`pickup_longitude` - the longitude where the meter was engaged

`pickup_latitude` - the latitude where the meter was engaged

`dropoff_longitude` - the longitude where the meter was disengaged

`dropoff_latitude` - the latitude where the meter was disengaged

`store_and_fwd_flag` - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip

`trip_duration` - duration of the trip in seconds

> Informations du fichier **train.csv**

In [ ]:
df.info()

In [ ]:
df.describe().T

> Informations du fichier **test.csv**

In [ ]:
df_test.info()

In [ ]:
df_test.describe().T

> On observe que certaines colonnes n'existent pas dans le fichier de test. La colonne `trip_duration` n'est évidemment pas présente puisqu'il s'agit de notre cible de prédiction. La colonne `dropoff_datetime` n'est pas non plus présente. Il faudra donc faire attention à ne pas la selectionner pour le choix des features utilisées.

## Data Visualisation

In [ ]:
plt.hist(df.loc[df.trip_duration<6000,"trip_duration"], bins=100);
plt.title('Répartition de voyage en taxi à NYC')
plt.xlabel('Durée d\'un voyage en taxi à NYC (sec)')
plt.ylabel('Nombre d\'enregistrement')
plt.show()

> On observe que la répartition des valeurs de la durée d'une course de taxi à NYC est réparti à droite. Nous pouvons donc essayer d'appliquer un logarithme sur ces valeurs afin d'avoir un meilleur aperçu.

In [ ]:
plt.hist(np.log(df.trip_duration), bins=200);
plt.title('Répartition de voyage en taxi à NYC (après utilisation d\'un logarithme sur les valeurs)')
plt.xlabel('Durée d\'un voyage en taxi à NYC')
plt.ylabel('Nombre d\'enregistrement')
plt.show()

In [ ]:
plt.figure()
g = sns.boxplot(x = 'vendor_id', y = 'trip_duration', data=df[df['trip_duration'] < 3000])
plt.title('Corrélation entre le vendeur et le temps de trajet d\'une course de taxi')
plt.xlabel('Identifiant du vendeur')
plt.ylabel('Temps de trajet d\'une course de taxi')
plt.show()

> On observe que la répartition du temps de trajet en est quasiment identique entre les deux entreprises. Je vais donc choisir de ne pas utiliser cette feature car elle n'est pas utile.

In [ ]:
plt.figure()
g = sns.boxplot(x = 'passenger_count', y = 'trip_duration', data=df[df['trip_duration'] < 3000])
plt.title('Corrélation entre le nombre de passager et le temps de trajet d\'une course de taxi')
plt.xlabel('Nombre de passager')
plt.ylabel('Temps de trajet d\'une course de taxi')
plt.show()

> On observe que la répartition du temps de trajet en est quasiment identique quel que soit le nombre de passager. Je vais donc choisir de ne pas utiliser cette feature car elle n'est pas utile.
De plus, on observe qu'il y a certaine course qui n'ont pas de passager, ce qui n'a pas vraiment de sens. Je vais donc choisir de supprimer les entrées qui ont 0 passager.

In [ ]:
df = df[df.passenger_count != 0]

## Data Preprocessing

### Missing & Duplicated values

In [ ]:
df.duplicated().sum()

> On observe qu'il y a 7 entrées dupliquées dans notre dataframe. Je vais choisir de les supprimer car même s'il est possible que 7 courses soient absolument identique, cela paraît peut probable.

In [ ]:
df = df.drop_duplicates()
df.duplicated().sum()

In [ ]:
df.isna().sum()

> Aucune valeur est indéfini, je n'apporte donc pas de traitement particulier.

### Outliers

In [ ]:
fig, ax = plt.subplots()
df.boxplot(['trip_duration'], fontsize=12)
fig.suptitle('Visualisation des outliers', fontsize=20)

> On observe qu'il y a certaines valeurs très extrèmes. Je considère qu'une course d'une durée inférieure à 1 minutes (60 secondes) et supérieure à 6 heures (21600 secondes) sont des outliers et donc méritent d'être supprimées

In [ ]:
df = df[(df['trip_duration'] > 60) & (df['trip_duration'] < 3600 * 6)]

### Gestion des variables catégoriques

> Pour la colonne `store_and_fwd_flag`, il est nécessaire de passer les valeurs en valeurs numériques si on veut que ce soit compris par notre modèle.

In [ ]:
df['store_and_fwd_flag'] = df['store_and_fwd_flag'].astype('category').cat.codes
df_test['store_and_fwd_flag'] = df_test['store_and_fwd_flag'].astype('category').cat.codes

## Features engineering

### Features Extraction

#### Travail sur les coordonnées : https://www.kaggle.com/gaborfodor/from-eda-to-the-top-lb-0-367
> Comme j'avais du mal à travailler avec les coordonnées, je me suis inspiré du travail du kernel "From EDA to the Top" qui fait un traitement des coordonnées avec PCA. PCA est un module de scikit learn qui permet de transformer des objets en objet de plus petite dimention. Ici cela va permettre d'avoir des données plus simple à interpreter pour notre modèle de prédiction.

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
coords = np.vstack((df[['pickup_latitude', 'pickup_longitude']].values,
                    df[['dropoff_latitude', 'dropoff_longitude']].values,
                    df_test[['pickup_latitude', 'pickup_longitude']].values,
                    df_test[['dropoff_latitude', 'dropoff_longitude']].values))

pca = PCA().fit(coords)

#Pour le fichier de train
df['pickup_pca0'] = pca.transform(df[['pickup_latitude', 'pickup_longitude']])[:, 0]
df['pickup_pca1'] = pca.transform(df[['pickup_latitude', 'pickup_longitude']])[:, 1]
df['dropoff_pca0'] = pca.transform(df[['dropoff_latitude', 'dropoff_longitude']])[:, 0]
df['dropoff_pca1'] = pca.transform(df[['dropoff_latitude', 'dropoff_longitude']])[:, 1]

#Pour le fichier de test
df_test['pickup_pca0'] = pca.transform(df_test[['pickup_latitude', 'pickup_longitude']])[:, 0]
df_test['pickup_pca1'] = pca.transform(df_test[['pickup_latitude', 'pickup_longitude']])[:, 1]
df_test['dropoff_pca0'] = pca.transform(df_test[['dropoff_latitude', 'dropoff_longitude']])[:, 0]
df_test['dropoff_pca1'] = pca.transform(df_test[['dropoff_latitude', 'dropoff_longitude']])[:, 1]

> Je me retrouve donc maintenant avec 4 nouvelles colonnes `pickup_pca0`, `pickup_pca1`, `dropoff_pca0` et `dropoff_pca1` qui correspondent à des données simplifiées de `pickup_latitude`, `pickup_longitude`, `dropoff_latitude` et `dropoff_longitude`.

In [ ]:
df.head(3)

#### Création des features `month`, `day` & `hour`

> Les dates sans traitement peuvent être difficile à interpréter. J'ai donc choisi d'extraire les valeurs intéressantes, c'est-à-dire le mois, le jour de la semaine et l'heure que j'extrais dans de nouvelles features.

In [ ]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])
df_test['pickup_datetime'] = pd.to_datetime(df_test['pickup_datetime'])

In [ ]:
df['hour'] = df.pickup_datetime.dt.hour
df['day'] = df.pickup_datetime.dt.dayofweek
df['month'] = df.pickup_datetime.dt.month
df_test['hour'] = df_test.pickup_datetime.dt.hour
df_test['day'] = df_test.pickup_datetime.dt.dayofweek
df_test['month'] = df_test.pickup_datetime.dt.month

#### Feature `distance`

> Pour la distance, j'ai choisi de faire un calcul simple qui correspond au calcul de la longueur d'un vecteur.

In [ ]:
df['distance2'] = np.sqrt((df['pickup_pca0']-df['dropoff_pca0'])**2
                        + (df['pickup_pca1']-df['dropoff_pca1'])**2)
df_test['distance2'] = np.sqrt((df_test['pickup_pca0']-df_test['dropoff_pca0'])**2
                        + (df_test['pickup_pca1']-df_test['dropoff_pca1'])**2)

#### Transformation logarithmique

In [ ]:
df['log_trip_duration'] = np.log(df['trip_duration'])

In [ ]:
df.head(3)

In [ ]:
df_test.head(3)

## Features selection

> Après plusieurs tests et déductions, j'ai choisi les features présentent ci-dessous. La target n'est maintenant plus `trip_duration` mais sa version log.

In [ ]:
NUM_VARS = ['pickup_pca0', 'pickup_pca1', 'dropoff_pca0', 'dropoff_pca1', 'month', 'hour', 'day', 'distance2']
TARGET = 'log_trip_duration'

In [ ]:
num_features = NUM_VARS

In [ ]:
X_train = df.loc[:, num_features]
y_train = df[TARGET]
X_test = df_test.loc[:, num_features]
X_train.shape, y_train.shape, X_test.shape

## Training

In [ ]:
from sklearn.ensemble import RandomForestRegressor

> Après plusieurs tests et déductions, j'ai choisi ces paramètres pour mon RandomForestRegressor.

In [ ]:
m = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, min_samples_split=15, max_features='auto', bootstrap=True)
m.fit(X_train, y_train)

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cv_scores = cross_val_score(m, X_train, y_train, cv=5, scoring='neg_mean_squared_log_error')
cv_scores

In [ ]:
y_test_pred = m.predict(X_test)
y_test_pred[:5]

In [ ]:
my_submission = pd.DataFrame({'id': df_test.id, 'trip_duration': np.exp(y_test_pred)})
my_submission.to_csv('submission.csv', index=False)

> Voici les 100 premières prédictions que mon modèle à généré.

In [ ]:
my_submission.head(100)